In [1]:
import networkx as nx
import ndlib.models.ModelConfig as mc
import ndlib.models.epidemics as ep
from ndlib.viz.mpl.DiffusionTrend import DiffusionTrend
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import csv
from tabulate import tabulate

no display found. Using non-interactive Agg backend


In [2]:
def give_mean(data):
    return np.mean(list(data.values()))

def give_properties(G):
    clustering_coefficient = give_mean(nx.clustering(G, nodes=None, weight=None))
    closeness_centrality = give_mean(nx.closeness_centrality(G, u=None, distance=None, wf_improved=True))
    betweenness_centrality = give_mean(nx.betweenness_centrality(G, k=None, normalized=True, weight=None, endpoints=False, seed=None))
    degree_centrality = give_mean(nx.degree_centrality(G))

    properties = np.matrix([clustering_coefficient, closeness_centrality, betweenness_centrality, degree_centrality])

    return(properties)

def evaluate(beta, gamma, I, n_iterations, G):
    """
    This function evaluates the spread of disease following the SIR model using the NDlib package 
    (https://ndlib.readthedocs.io/en/latest/index.html). It first implements the SIR model on the given network G. Next,
    the model parameters beta, gamma, and the number of initial infections is set for the model. The returned dictionairy trends gives the 
    amount of nodes in each state and the rate of change for each state. 
    """
    model = ep.SIRModel(G)

    # Setting model parameters
    cfg = mc.Configuration()
    cfg.add_model_parameter('beta', beta)
    cfg.add_model_parameter('gamma', gamma)
    cfg.add_model_parameter("fraction_infected", I)

    model.set_initial_status(cfg)

    # Running the simulation
    iterations = model.iteration_bunch(n_iterations)
    trends = model.build_trends(iterations)

    return trends

In [15]:
# Generate Networks of equivalent Form
p = 0.1
runs=1000

def give_networks(n):
    k = int(p*n)
    m = int(k/2)

    properties_er, properties_ws, properties_ba = np.zeros((4,runs)), np.zeros((4,runs)), np.zeros((4,runs))
    er_list, ws_list, ba_list = [], [], []

    for i in range(0, runs):
        seed=int(i)
        er = nx.erdos_renyi_graph(n, p, seed)   
        properties_er[:,i] = give_properties(er)
        er_list.append(er)
        
    for i in range(0, runs):
        seed=int(i)
        ws = nx.watts_strogatz_graph(n, k, p, seed)
        properties_ws[:,i] = give_properties(ws)
        ws_list.append(ws)

    for i in range(0, runs):
        seed=int(i)
        ba = nx.barabasi_albert_graph(n, m, seed)
        properties_ba[:,i] = give_properties(ba)
        ba_list.append(ba)

    table_sd = [['barabasi-albert', 3*np.std(properties_ba[0,:]), 3*np.std(properties_ba[1,:]), 3*np.std(properties_ba[2,:]), 3*np.std(properties_ba[3,:])],
            ['watts-strogatz', 3*np.std(properties_ws[0,:]), 3*np.std(properties_ws[1,:]), 3*np.std(properties_ws[2,:]), 3*np.std(properties_ws[3,:])], 
            ['erdos-renyi', 3*np.std(properties_er[0,:]), 3*np.std(properties_er[1,:]), 3*np.std(properties_er[2,:]), 3*np.std(properties_er[3,:])]]

    table = [['barabasi-albert', np.mean(properties_ba[0,:]), np.mean(properties_ba[1,:]), np.mean(properties_ba[2,:]), np.mean(properties_ba[3,:])],
            ['watts-strogatz', np.mean(properties_ws[0,:]), np.mean(properties_ws[1,:]), np.mean(properties_ws[2,:]), np.mean(properties_ws[3,:])], 
            ['erdos-renyi', np.mean(properties_er[0,:]), np.mean(properties_er[1,:]), np.mean(properties_er[2,:]), np.mean(properties_er[3,:])]]

    print(f'\n 3\u03C3 for n={n} nodes')
    print(tabulate(table_sd, headers=['clustering_coefficient', 'closeness_centrality', 'betweenness_centrality', 'degree_centrality']))

    # print(f'\n properties for n={n} nodes')
    # print(tabulate(table, headers=['clustering_coefficient', 'closeness_centrality', 'betweenness_centrality', 'degree_centrality']))

    return (er_list, ws_list, ba_list)

er_list_50, ws_list_50, ba_list_50 = give_networks(50)
er_list, ws_list, ba_list = give_networks(100) ### Saving for next experiment
er_list_200, ws_list_200, ba_list_200 = give_networks(200)


 3σ for n=50 nodes
                   clustering_coefficient    closeness_centrality    betweenness_centrality    degree_centrality
---------------  ------------------------  ----------------------  ------------------------  -------------------
barabasi-albert                 0.187016                0.0332018                0.00469626          8.10301e-17
watts-strogatz                  0.121034                0.0640531                0.0225923           4.16334e-17
erdos-renyi                     0.0828191               0.0600728                0.00766563          0.0248683

 3σ for n=100 nodes
                   clustering_coefficient    closeness_centrality    betweenness_centrality    degree_centrality
---------------  ------------------------  ----------------------  ------------------------  -------------------
barabasi-albert                 0.0605914               0.0103555               0.000508693          2.42405e-17
watts-strogatz                  0.0646079               0

In [5]:
# Simulate SIR spread on the network
plt.clf()
beta_list =  [1, 1/3, 1/6] 
gamma = 1/3
I0 = 0.01
n_iterations = 30
n=100

for beta in beta_list:
    er_X, er_Y, er_Z = [0]*n_iterations, [0]*n_iterations, [0]*n_iterations
    ws_X, ws_Y, ws_Z = [0]*n_iterations, [0]*n_iterations, [0]*n_iterations
    ba_X, ba_Y, ba_Z = [0]*n_iterations, [0]*n_iterations, [0]*n_iterations

    for i in range(n):
        er = er_list[i]
        ws = ws_list[i]
        ba = ba_list[i]

        trends_er = evaluate(beta, gamma, I0, n_iterations, er)
        trends_ws = evaluate(beta, gamma, I0, n_iterations, ws)
        trends_ba = evaluate(beta, gamma, I0, n_iterations, ba)

        # Add values of next network evaluation
        er_X = np.add(er_X, trends_er[0]['trends']['node_count'][0])
        er_Y = np.add(er_Y, trends_er[0]['trends']['node_count'][1])
        er_Z = np.add(er_Z, trends_er[0]['trends']['node_count'][2])

        ws_X = np.add(ws_X, trends_ws[0]['trends']['node_count'][0]) 
        ws_Y = np.add(ws_Y, trends_ws[0]['trends']['node_count'][1]) 
        ws_Z = np.add(ws_Z, trends_ws[0]['trends']['node_count'][2]) 

        ba_X = np.add(ba_X, trends_ba[0]['trends']['node_count'][0]) 
        ba_Y = np.add(ba_Y, trends_ba[0]['trends']['node_count'][1]) 
        ba_Z = np.add(ba_Z, trends_ba[0]['trends']['node_count'][2]) 

    er_I = er_Y/(runs*n)
    ws_I = ws_Y/(runs*n)
    ba_I = ba_Y/(runs*n)

    plt.figure(figsize=(10,6), tight_layout=True)
    plt.ylim(0,1)

    
    plt.plot(er_I, label="Erd\u0151s-R\u00e9nyi", linewidth=2)
    plt.plot(ws_I, label="Watts-Strogatz", linewidth=2)
    plt.plot(ba_I, label="Barab\u00e1si-Albert", linewidth=2)
    plt.legend()

    plt.xlabel('t', fontsize = 25)
    plt.ylabel('I', fontsize = 25)
    plt.xticks(fontsize=24)
    plt.yticks(fontsize=24)
    plt.title(f'Fraction of infected hosts over time, \u03B2={np.round(beta,2)}', fontsize=25)
    plt.legend(fontsize = 24)

    plt.savefig(f'plots2/beta-{np.round(beta,2)}_gamma-{np.round(gamma,2)}_I0-{I0}.png')

    plt.clf()

In [30]:
# Dynamic vaccination campaign

def evaluate_vacc_step(beta, gamma, infected_nodes_initial, n_iterations, G, test_accuracy=1, tests_used=0, n_vacc=0, test_max=200):
    infected_nodes = infected_nodes_initial
    model = ep.SIRModel(G) 

    if len(infected_nodes) != 0 and tests_used < test_max:
        for _ in range(n_vacc):
            closeness_centrality = nx.closeness_centrality(G)
            closeness_centrality_values = list(closeness_centrality.values())
            closeness_centrality_ordered = sorted(list(closeness_centrality.values()), reverse=True)

            infected = True
            next_max = 0

            while infected == True and next_max < len(closeness_centrality_ordered):
                accuracy = np.random.binomial(1, test_accuracy)
                target_index = closeness_centrality_values.index(closeness_centrality_ordered[next_max])
                target_node = list(closeness_centrality.keys())[target_index]
                tests_used += 1
                next_max += 1

                if target_node in infected_nodes and accuracy == 0:
                    infected = False
                
                if target_node not in infected_nodes and accuracy == 1 and tests_used < test_max:
                    G.remove_node(target_node)
                    infected = False
                
    for _ in range(n_iterations):

        recovered_nodes = []
        
        # Model Configuration
        cfg = mc.Configuration()
        cfg.add_model_parameter('beta', beta)
        cfg.add_model_parameter('gamma', gamma)
        cfg.add_model_initial_configuration("Infected", infected_nodes)


        model.set_initial_status(cfg)

        # Simulation execution
        iterations = model.iteration_bunch(1)
        trends = model.build_trends(iterations)

        # Find direct neighbors of infected nodes
        neighbors_infected_nodes = []
        for node in infected_nodes:
            # print(node)
        
            for neighbor_node in G.neighbors(node):
                neighbors_infected_nodes.append(neighbor_node)

            # remove neighbor if neighbor already infected
            overlap = np.intersect1d(neighbors_infected_nodes, infected_nodes)
            for node in overlap:
                neighbors_infected_nodes.remove(node)
            
            # only have each neighbour once, even if it is shared by multiple nodes
            neighbors_infected_nodes = list(np.unique(neighbors_infected_nodes))

        number_of_infections = trends[0]['trends']['status_delta'][1][0]

        
        if number_of_infections > 0:
            """Random neighbors of the infected nodes are now infected, if the simulation asks for more infections than
            there are direct neighbors all direct neuighbors will be infected"""
            if len(neighbors_infected_nodes) > number_of_infections:
                added_infections = np.random.choice(neighbors_infected_nodes, number_of_infections, replace=False)
            else:
                added_infections = np.random.choice(neighbors_infected_nodes, len(neighbors_infected_nodes), replace=False)
            
            infected_nodes.extend(added_infections)
        
        if number_of_infections < 0:
            """Random infected nodes are now recovered, if the simulation asks for more recovering infected hosts than
            there are infected hosts all infected hosts will recover"""
            if len(infected_nodes) > np.abs(number_of_infections):
                removed_infections = np.random.choice(infected_nodes, np.abs(number_of_infections), replace=False)
            else:
                removed_infections = np.random.choice(infected_nodes, len(infected_nodes), replace=False)

            for removed_infection in removed_infections:
                infected_nodes.remove(removed_infection)
                G.remove_node(removed_infection)
                recovered_nodes.append(removed_infection)

    return G, infected_nodes, recovered_nodes, tests_used
    

"Creating edgelist"
data = np.loadtxt(open("transmission_network.csv", "rb"), delimiter=";", skiprows=1)[:, 1:]
nonzero_x = np.nonzero(data)[0]
nonzero_y = np.nonzero(data)[1]

edges=[]
for i in range(len(nonzero_x)):
    edges.append((nonzero_x[i], nonzero_y[i]))

"Set parameters"
beta =  1
gamma = 1/3
n_iterations = 2
n_vacc_list = [1,3,5,10]
test_accuracy_list=[0.25,0.5,1]

n_samples=1000
end_time={}
end_time_std={}
max_infected={}
max_infected_std={}

for test_accuracy in test_accuracy_list:
    for n_vacc in n_vacc_list:
        end_time_list = []
        max_infected_list = []
        for _ in range(n_samples):
            G = nx.from_edgelist(edges)
            infected_nodes =  list(np.random.choice(list(G.nodes), 5, replace=False))
            tests_used = 0
            t = -1 ### Start at t=0
            test_max = 0
            max_infected_nodes = 0
            while len(infected_nodes) != 0:
                t+=1
                G_new, infected_nodes_new, recovered_nodes, tests_used_new = evaluate_vacc_step(beta, gamma, infected_nodes, n_iterations, G, test_accuracy, tests_used, n_vacc, test_max)
                G=G_new
                infected_nodes=infected_nodes
                tests_used = tests_used_new

                if len(infected_nodes) > max_infected_nodes:
                    max_infected_nodes = len(infected_nodes)

            end_time_list.append(t)
            max_infected_list.append(max_infected_nodes)

        end_time[test_accuracy, n_vacc] = np.mean(end_time_list)
        max_infected[test_accuracy, n_vacc] = np.mean(max_infected_list)

        end_time_std[test_accuracy, n_vacc] = np.std(end_time_list)
        max_infected_std[test_accuracy, n_vacc] = np.std(max_infected_list)


In [31]:
### Final run for the case of no infecteds 

end_time_value_list = []
max_infected_nodes_list = []
for _ in range(n_samples):
    G = nx.from_edgelist(edges)
    infected_nodes =  list(np.random.choice(list(G.nodes), 5, replace=False))
    tests_used = 0
    t = -1 ### Start at t=0
    test_max = 0
    max_infected_nodes = 0
    while len(infected_nodes) != 0:
        t+=1
        G_new, infected_nodes_new, recovered_nodes, tests_used_new = evaluate_vacc_step(beta, gamma, infected_nodes, n_iterations, G, test_accuracy, tests_used, n_vacc, test_max)
        G=G_new
        infected_nodes=infected_nodes
        tests_used = tests_used_new

        if len(infected_nodes) > max_infected_nodes:
            max_infected_nodes = len(infected_nodes)

    end_time_value_list.append(t)
    max_infected_nodes_list.append(max_infected_nodes)

print('Average time until the disease passes', np.mean(end_time_value_list), 3*np.std(end_time_value_list))
print('Average maximum number of infected hosts', np.mean(max_infected_nodes_list), 3*np.std(max_infected_nodes_list))

Average time until the disease passes 74.163 31.072880120774126
Average maximum number of infected hosts 267.11 18.724398521714924


In [32]:
table = [[test_accuracy_list[0], end_time[test_accuracy_list[0], n_vacc_list[0]], end_time[test_accuracy_list[0], n_vacc_list[1]], end_time[test_accuracy_list[0], n_vacc_list[2]], end_time[test_accuracy_list[0], n_vacc_list[3]]],
         [test_accuracy_list[1], end_time[test_accuracy_list[1], n_vacc_list[0]], end_time[test_accuracy_list[1], n_vacc_list[1]], end_time[test_accuracy_list[1], n_vacc_list[2]], end_time[test_accuracy_list[1], n_vacc_list[3]]], 
         [test_accuracy_list[2], end_time[test_accuracy_list[2], n_vacc_list[0]],end_time[test_accuracy_list[2], n_vacc_list[1]], end_time[test_accuracy_list[2], n_vacc_list[2]], end_time[test_accuracy_list[2], n_vacc_list[3]]]]

table_std = [[test_accuracy_list[0], 3*end_time_std[test_accuracy_list[0], n_vacc_list[0]], 3*end_time_std[test_accuracy_list[0], n_vacc_list[1]], 3*end_time_std[test_accuracy_list[0], n_vacc_list[2]], 3*end_time_std[test_accuracy_list[0], n_vacc_list[3]]],
         [test_accuracy_list[1], 3*end_time_std[test_accuracy_list[1], n_vacc_list[0]], 3*end_time_std[test_accuracy_list[1], n_vacc_list[1]], 3*end_time_std[test_accuracy_list[1], n_vacc_list[2]], 3*end_time_std[test_accuracy_list[1], n_vacc_list[3]]], 
         [test_accuracy_list[2], 3*end_time_std[test_accuracy_list[2], n_vacc_list[0]],3*end_time_std[test_accuracy_list[2], n_vacc_list[1]], 3*end_time_std[test_accuracy_list[2], n_vacc_list[2]], 3*end_time_std[test_accuracy_list[2], n_vacc_list[3]]]]

print('Average time until the disease passes')
print(tabulate(table, headers=n_vacc_list))

print('3\u03C3 on endtime')
print(tabulate(table_std, headers=n_vacc_list))

Average time until the disease passes
           1       3       5      10
----  ------  ------  ------  ------
0.25  73.938  75.51   74.354  74.381
0.5   74.574  73.863  74.287  73.899
1     74.742  74.481  74.833  74.4
3σ on endtime
            1        3        5       10
----  -------  -------  -------  -------
0.25  31.2245  29.6144  30.8012  30.5632
0.5   30.1052  29.0186  30.142   29.6983
1     30.6851  31.2256  31.4083  29.9417


In [29]:
table = [[test_accuracy_list[0], max_infected[test_accuracy_list[0], n_vacc_list[0]], max_infected[test_accuracy_list[0], n_vacc_list[1]], max_infected[test_accuracy_list[0], n_vacc_list[2]], max_infected[test_accuracy_list[0], n_vacc_list[3]]],
         [test_accuracy_list[1], max_infected[test_accuracy_list[1], n_vacc_list[0]], max_infected[test_accuracy_list[1], n_vacc_list[1]], max_infected[test_accuracy_list[1], n_vacc_list[2]], max_infected[test_accuracy_list[1], n_vacc_list[3]]], 
         [test_accuracy_list[2], max_infected[test_accuracy_list[2], n_vacc_list[0]],max_infected[test_accuracy_list[2], n_vacc_list[1]], max_infected[test_accuracy_list[2], n_vacc_list[2]], max_infected[test_accuracy_list[2], n_vacc_list[3]]]]

table_std = [[test_accuracy_list[0], 3*max_infected_std[test_accuracy_list[0], n_vacc_list[0]], 3*max_infected_std[test_accuracy_list[0], n_vacc_list[1]], 3*max_infected_std[test_accuracy_list[0], n_vacc_list[2]], 3*max_infected_std[test_accuracy_list[0], n_vacc_list[3]]],
         [test_accuracy_list[1], 3*max_infected_std[test_accuracy_list[1], n_vacc_list[0]], 3*max_infected_std[test_accuracy_list[1], n_vacc_list[1]], 3*max_infected_std[test_accuracy_list[1], n_vacc_list[2]], 3*max_infected_std[test_accuracy_list[1], n_vacc_list[3]]], 
         [test_accuracy_list[2], 3*max_infected_std[test_accuracy_list[2], n_vacc_list[0]], 3*max_infected_std[test_accuracy_list[2], n_vacc_list[1]], 3*max_infected_std[test_accuracy_list[2], n_vacc_list[2]], 3*max_infected_std[test_accuracy_list[2], n_vacc_list[3]]]]

print('Average maximum number of infected hosts')
print(tabulate(table, headers=n_vacc_list))

print('3\u03C3 on max infected')
print(tabulate(table_std, headers=n_vacc_list))

Average maximum number of infected hosts
           1       3       5      10
----  ------  ------  ------  ------
0.25  267.23  266.81  267.45  266.53
0.5   266.83  266.8   266.61  266.58
1     266.56  267.24  265.74  266.74
3σ on max infected
            1        3        5       10
----  -------  -------  -------  -------
0.25  19.7219  19.6755  18.0219  21.7265
0.5   18.6099  19.5668  16.9275  16.6557
1     17.0833  17.9305  17.8978  17.7615
